# Limpieza de tablas

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
path_originales = "originales"
os.listdir(path_originales)

['circuits.csv',
 'constructors.csv',
 'constructor_results.csv',
 'constructor_standings.csv',
 'drivers.csv',
 'driver_standings.csv',
 'lap_times.csv',
 'pit_stops.csv',
 'qualifying.csv',
 'races.csv',
 'results.csv',
 'seasons.csv',
 'sprint_results.csv',
 'status.csv']

# Tablas de catálogos

## Circuitos
Variables y tipo de variable en la tabla.

**Observaciones**:
* La tabla original cuenta con una variable `alt: altitud`, que debería ser entera. Sin embargo, tiene valores nulos identificados como `\\N`, estos serán remplazados por `np.nan` para todas las tablas de la BD

In [3]:
circuits = pd.read_csv(os.path.join(path_originales, 'circuits.csv'))

#Se reemplaza \\N por np.nan
circuits.replace(to_replace='\\N', value=np.nan, inplace=True)

#Se convierte columna de altitud a numérica
circuits['alt'] = circuits['alt'].astype('float')

circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130.0,http://en.wikipedia.org/wiki/Istanbul_Park


In [4]:
circuits.dtypes

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt           float64
url            object
dtype: object

In [5]:
circuits.describe()

,circuitId,lat,lng,alt
count,76.000000,76.000000,76.000000,74.000000
mean,39.355263,33.407770,2.606284,248.189189
std,22.680214,22.958321,64.553438,365.041851
min,1.000000,-37.849700,-118.189000,-7.000000
25%,19.750000,32.417775,-9.250838,19.250000
50%,39.500000,41.061100,4.128885,129.500000
75%,58.250000,47.020150,21.404850,315.500000
max,79.000000,57.265300,144.968000,2227.000000


**Acerca de variables numéricas**:
* Tanto `circuitId`, como `lat`, y `lng` están completas.
* `alt` tiene 2 valores faltantes, mismo que se pueden obtener de `url`

In [6]:
print(f"Valores faltantes en 'alt': {circuits['alt'].isna().sum()}\n")

Valores faltantes en 'alt': 2



In [7]:
#Se obtienen ID's de los circuitos faltantes
circuits.loc[circuits['alt'].isna()]

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
74,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,NaN,http://en.wikipedia.org/wiki/Losail_Internatio...
75,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,NaN,http://en.wikipedia.org/wiki/Miami_Internation...


La altura de los circuitos faltantes se obtiene de la información geográfica de las ciudades en las que se corren el GP:
* **Losail International Circuit**, cercano a Doha, Catar. **13 m** de altura, [referencia](https://dateandtime.info/es/citycoordinates.php?id=290030#:~:text=Coordenadas%20geogr%C3%A1ficas%20de%20Doha%2C%20Catar%20Latitud%3A%2025%C2%B016%E2%80%B245%E2%80%B3%20N,Altitud%20sobre%20el%20nivel%20del%20mar%3A%2013%20m)
* **Miami International Autodrome**, cercano al Hard Rock Stadium de los Miami Dolphins. **3m** de altura, [referencia](https://elevation.maplogs.com/poi/hard_rock_stadium_don_shula_dr_miami_gardens_fl_usa.416642.html)

In [8]:
# Se completan valores de altitud
circuits.loc[74, 'alt'] = 13
circuits.loc[75, 'alt'] = 3

**Finalmente**:
<br>Se quita la columns `url` de lo que será la tabla de trabajo, y se guarda una copia.

In [9]:
circuits.drop(columns='url', inplace=True)
circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130.0


**IMPORTANTE**:
Esta tabla no se almacena debido a que su información está contenida dentro de la tabla `races` más adelante

## Constructores

In [10]:
constructors = pd.read_csv(os.path.join(path_originales, 'constructors.csv'))

#Se reemplaza \\N por np.nan
constructors.replace(to_replace='\\N', value=np.nan, inplace=True)

constructors.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [11]:
constructors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
 3   nationality     211 non-null    object
 4   url             211 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


La tabla de constructores está completa

**Finalmente**:
<br>Se quita la columns `url` de lo que será la tabla de trabajo, y se guarda una copia

In [12]:
constructors.drop(columns='url', inplace=True)
constructors.to_csv('constructors.csv', index=False)

constructors.head()

,constructorId,constructorRef,name,nationality
0,1,mclaren,McLaren,British
1,2,bmw_sauber,BMW Sauber,German
2,3,williams,Williams,British
3,4,renault,Renault,French
4,5,toro_rosso,Toro Rosso,Italian


## Pilotos

In [13]:
drivers = pd.read_csv(os.path.join(path_originales, 'drivers.csv'))

#Se reemplaza \\N por np.nan
drivers.replace(to_replace='\\N', value=np.nan, inplace=True)

#Se convierte la fecha de nacimiento a variable tipo date
drivers['dob'] = pd.to_datetime(drivers['dob'])

drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [14]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     854 non-null    int64         
 1   driverRef    854 non-null    object        
 2   number       51 non-null     object        
 3   code         97 non-null     object        
 4   forename     854 non-null    object        
 5   surname      854 non-null    object        
 6   dob          854 non-null    datetime64[ns]
 7   nationality  854 non-null    object        
 8   url          854 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 60.2+ KB


**Observaciones**:
* A lo largo de la historia han habido 854 pilotos
* Solamente se tiene referencia de número para 51 de ellos
* Solo 97 pilotos cuentan con un código de identificación de 3 letras

No es relevante completar estas columnas debido a que son una referencia única adicional para cada piloto, para dicho propósito se puede utilizar la variable `driverRef`. Dicho eso, se descatan columnas `number`, `code`, y `url`

In [15]:
drivers.drop(columns=['number', 'code', 'url'], inplace=True)
drivers.to_csv('drivers.csv', index=False)

drivers.head()

,driverId,driverRef,forename,surname,dob,nationality
0,1,hamilton,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,Nico,Rosberg,1985-06-27,German
3,4,alonso,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,Heikki,Kovalainen,1981-10-19,Finnish


# Tablas con llaves foráneas

**Observaciones**:
* Existe una tabla con llave foránea que destaca sobre todas las demás, y es la de `races`. Destaca debido a que es una tabla utilizada en la mayoría de las demás

## Carreras

In [16]:
races = pd.read_csv(os.path.join(path_originales, 'races.csv'))

#Se reemplaza \\N por np.nan
races.replace(to_replace='\\N', value=np.nan, inplace=True)

races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Las columnas que empiezas con los sufijo `date`, `time` hacen referencia a la fecha y hora en la que se celebraron las prácticas libres, sesiones de clasificación, y sesiones sprint de las diferentes carreras. Sin embargo, no aportan información adicional para propósitos del análisis que se desea realizar. Por esta razón se descartan.

Al igual que en las tablas anteriores, se descarta columna de `url`

In [17]:
#Se descartan variables con sufijo _date
drop_date = races.filter(regex='_date', axis=1).columns.to_list()
races.drop(columns=drop_date, inplace=True)

#Se descartan variables con sufijo _time
drop_time = races.filter(regex='_time', axis=1).columns.to_list()
races.drop(columns=drop_time, inplace=True)

#Se descarta url
races.drop(columns='url', inplace=True)

In [18]:
#Se convierte la fecha y hora  de carrera a variables tipo date y time
races['date'] = pd.to_datetime(races['date'])
races['time'] = pd.to_datetime(races['time']).dt.time

In [19]:
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   raceId     1079 non-null   int64         
 1   year       1079 non-null   int64         
 2   round      1079 non-null   int64         
 3   circuitId  1079 non-null   int64         
 4   name       1079 non-null   object        
 5   date       1079 non-null   datetime64[ns]
 6   time       348 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 59.1+ KB


**Finalmente**:
<br> Se combina la tabla de carreras con la tabla de circuitos por la izquierda.

In [20]:
races = pd.merge(left=races,
        right=circuits,
        how='left',
        on='circuitId',
        suffixes=('Race', 'Circuit'))

Se descarta `circuitId` de tabla de races debido a que es la única tabla que lo utiliza

In [21]:
races.drop(columns='circuitId', inplace=True)
races.to_csv('races.csv', index=False)

races.head()

,raceId,year,round,nameRace,date,time,circuitRef,nameCircuit,location,country,lat,lng,alt
0,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0
1,2,2009,2,Malaysian Grand Prix,2009-04-05,09:00:00,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0
2,3,2009,3,Chinese Grand Prix,2009-04-19,07:00:00,shanghai,Shanghai International Circuit,Shanghai,China,31.33890,121.22000,5.0
3,4,2009,4,Bahrain Grand Prix,2009-04-26,12:00:00,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0
4,5,2009,5,Spanish Grand Prix,2009-05-10,12:00:00,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0


## Dependientes de `drivers` y `races`

### Paradas en Pits
**Observaciones**:
* La variable `duration` pretende definir la duración de la parada en segundos. Sin embargo, hay registros de paradas de más de 1 minuto lo cual hace que la transformación de esta columna se complique
* Afortunadamente, la información de `duration` también está contenida en la variable `milliseconds`

In [22]:
pit_stops = pd.read_csv(os.path.join(path_originales, 'pit_stops.csv'))

#Se reemplaza \\N por np.nan
pit_stops.replace(to_replace='\\N', value=np.nan, inplace=True)

pit_stops.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [23]:
#Se convierte time a variable tipo time
pit_stops['time'] = pd.to_datetime(pit_stops['time']).dt.time

In [24]:
#Se descarta duration
pit_stops.drop(columns='duration', inplace=True)

In [25]:
pit_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9299 entries, 0 to 9298
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        9299 non-null   int64 
 1   driverId      9299 non-null   int64 
 2   stop          9299 non-null   int64 
 3   lap           9299 non-null   int64 
 4   time          9299 non-null   object
 5   milliseconds  9299 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 436.0+ KB


In [26]:
pit_stops[['raceId', 'driverId', 'stop', 'milliseconds']].groupby(by=['raceId', 'driverId']).sum()

stop  milliseconds
raceId driverId                    
841    1            3         46426
       2            3         48092
       3            1         23716
       4            6         72165
       5            1         24865
...               ...           ...
1086   848          6         77338
       849          6         73094
       852          6         65721
       854          3         45519
       855          3         50539

[4535 rows x 2 columns]

In [27]:
results = pd.read_csv(os.path.join(path_originales, 'results.csv'))

#Se reemplaza \\N por np.nan
results.replace(to_replace='\\N', value=np.nan, inplace=True)

#Se convierte columna de altitud a numérica
results['number'] = results['number'].astype('float')
results['position'] = results['position'].astype('float')
results['fastestLap'] = results['fastestLap'].astype('float')
results['rank'] = results['rank'].astype('float')
results['fastestLapSpeed'] = results['fastestLapSpeed'].astype('float')

#Se descartan variables
results.drop(columns=['positionText'], inplace=True)

results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,10.0,58,1:34:50.616,5690616,39.0,2.0,1:27.452,218.300,1
1,2,18,2,2,3.0,5,2.0,2,8.0,58,+5.478,5696094,41.0,3.0,1:27.739,217.586,1
2,3,18,3,3,7.0,7,3.0,3,6.0,58,+8.163,5698779,41.0,5.0,1:28.090,216.719,1
3,4,18,4,4,5.0,11,4.0,4,5.0,58,+17.181,5707797,58.0,7.0,1:28.603,215.464,1
4,5,18,5,1,23.0,3,5.0,5,4.0,58,+18.014,5708630,43.0,1.0,1:27.418,218.385,1


In [28]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25660 entries, 0 to 25659
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25660 non-null  int64  
 1   raceId           25660 non-null  int64  
 2   driverId         25660 non-null  int64  
 3   constructorId    25660 non-null  int64  
 4   number           25654 non-null  float64
 5   grid             25660 non-null  int64  
 6   position         14833 non-null  float64
 7   positionOrder    25660 non-null  int64  
 8   points           25660 non-null  float64
 9   laps             25660 non-null  int64  
 10  time             6964 non-null   object 
 11  milliseconds     6963 non-null   object 
 12  fastestLap       7206 non-null   float64
 13  rank             7411 non-null   float64
 14  fastestLapTime   7206 non-null   object 
 15  fastestLapSpeed  7206 non-null   float64
 16  statusId         25660 non-null  int64  
dtypes: float64(6

In [29]:
sum(results['position'] == results['positionOrder'])

14808

In [30]:
status = pd.read_csv(os.path.join(path_originales, 'status.csv'))

#Se reemplaza \\N por np.nan
status.replace(to_replace='\\N', value=np.nan, inplace=True)

status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   statusId  139 non-null    int64 
 1   status    139 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


# Lap times

In [31]:
lap_times = pd.read_csv(os.path.join(path_originales, 'lap_times.csv'))
lap_times.head(20)

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
5,841,20,6,1,1:32.605,92605
6,841,20,7,1,1:32.502,92502
7,841,20,8,1,1:32.537,92537
8,841,20,9,1,1:33.240,93240
9,841,20,10,1,1:32.572,92572


In [32]:
lap_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528785 entries, 0 to 528784
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        528785 non-null  int64 
 1   driverId      528785 non-null  int64 
 2   lap           528785 non-null  int64 
 3   position      528785 non-null  int64 
 4   time          528785 non-null  object
 5   milliseconds  528785 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 24.2+ MB


In [33]:
lap_times.sort_values(by=['raceId','driverId','lap'], inplace=True, ignore_index=True)
lap_times.head(20)

,raceId,driverId,lap,position,time,milliseconds
0,1,1,1,13,1:49.088,109088
1,1,1,2,12,1:33.740,93740
2,1,1,3,11,1:31.600,91600
3,1,1,4,10,1:31.067,91067
4,1,1,5,10,1:32.129,92129
5,1,1,6,9,1:30.469,90469
6,1,1,7,9,1:29.488,89488
7,1,1,8,9,1:30.302,90302
8,1,1,9,9,1:30.889,90889
9,1,1,10,8,1:32.418,92418


In [34]:
lap_pivot = lap_times.pivot(index=['raceId','driverId'], columns='lap', values='milliseconds')
lap_pivot.reset_index(inplace=True)
lap_pivot

lap,raceId,driverId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87
0,1,1,109088.0,93740.0,91600.0,91067.0,92129.0,90469.0,89488.0,90302.0,90889.0,92418.0,115549.0,98029.0,92372.0,91749.0,90771.0,90751.0,90984.0,95580.0,101892.0,105445.0,98683.0,153309.0,152352.0,136183.0,96863.0,92198.0,91291.0,90377.0,90077.0,89912.0,89743.0,90027.0,89560.0,89723.0,89474.0,89894.0,89422.0,89104.0,89020.0,90261.0,89824.0,89484.0,107494.0,95135.0,89444.0,90224.0,90528.0,90147.0,91175.0,90082.0,89611.0,89722.0,89599.0,91129.0,90149.0,107829.0,134442.0,150895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,157754.0,97640.0,91697.0,91867.0,92234.0,92036.0,91659.0,91801.0,93491.0,93063.0,92439.0,91452.0,91523.0,91408.0,91232.0,91112.0,91209.0,125300.0,121930.0,108094.0,108456.0,95396.0,121351.0,135301.0,97154.0,93808.0,93104.0,92280.0,92356.0,91390.0,91188.0,92289.0,91734.0,90967.0,90959.0,90756.0,90685.0,90269.0,89941.0,90031.0,89982.0,89950.0,89748.0,107500.0,94324.0,89421.0,88968.0,88283.0,88373.0,88697.0,89476.0,91178.0,91204.0,91444.0,91730.0,112621.0,128962.0,132652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,102659.0,92091.0,91148.0,89660.0,89468.0,89643.0,90181.0,91501.0,92315.0,90219.0,93984.0,91984.0,88519.0,88428.0,88553.0,120177.0,97629.0,93148.0,103569.0,102455.0,105705.0,155699.0,151994.0,141534.0,98511.0,91336.0,91444.0,90439.0,90328.0,90032.0,89822.0,89350.0,89152.0,89059.0,89294.0,89878.0,89801.0,89737.0,89748.0,90112.0,90626.0,88957.0,88728.0,106312.0,95294.0,89168.0,88506.0,87706.0,88803.0,90277.0,91532.0,91400.0,94441.0,96161.0,93169.0,123117.0,116092.0,150911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,109367.0,94983.0,92175.0,93114.0,91831.0,91556.0,91683.0,91290.0,91083.0,91569.0,90944.0,90522.0,90503.0,90914.0,90718.0,90863.0,90746.0,92498.0,131126.0,111011.0,95702.0,153045.0,152290.0,135567.0,97840.0,91912.0,91215.0,90771.0,90595.0,90132.0,89646.0,89993.0,89684.0,89466.0,89714.0,89539.0,89778.0,89744.0,90436.0,91106.0,90373.0,90751.0,90230.0,90181.0,90160.0,90003.0,90115.0,89933.0,89695.0,89451.0,107303.0,96483.0,88712.0,88724.0,89171.0,124117.0,118145.0,150445.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,6,104899.0,92734.0,92684.0,90952.0,90152.0,89923.0,90057.0,90173.0,90965.0,92313.0,90804.0,91343.0,90955.0,91152.0,90596.0,90558.0,90718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,1086,848,105881.0,109840.0,113951.0,85048.0,84719.0,84924.0,85177.0,85329.0,85360.0,85585.0,85713.0,85300.0,85510.0,85679.0,85862.0,85860.0,86941.0,87336.0,87467.0,86663.0,90341.0,104533.0,85202.0,84805.0,84778.0,84677.0,85259.0,85113.0,84750.0,85000.0,85290.0,84491.0,90203.0,86887.0,86963.0,85930.0,85408.0,85095.0,84920.0,87273.0,104871.0,83357.0,83047.0,83430.0,83591.0,83866.0,83891.0,83857.0,83992.0,83637.0,86056.0,84083.0,84621.0,85927.0,85544.0,84514.0,85166.0,85248.0,85137.0,85062.0,85086.0,85340.0,85460.0,85785.0,86377.0,94127.0,109545.0,8

In [35]:
laps = lap_pivot.columns.to_list()
laps.remove('raceId')
laps.remove('driverId')
#laps

In [36]:
for lap in laps:
    if lap == laps[-1]:
        pass
    else:
        columna = f"delta_{lap+1}"
        lap_pivot[columna] = lap_pivot[lap+1] - lap_pivot[lap]
    
lap_pivot

lap,raceId,driverId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,delta_2,delta_3,delta_4,delta_5,delta_6,delta_7,delta_8,delta_9,delta_10,delta_11,delta_12,delta_13,delta_14,delta_15,delta_16,delta_17,delta_18,delta_19,delta_20,delta_21,delta_22,delta_23,delta_24,delta_25,delta_26,delta_27,delta_28,delta_29,delta_30,delta_31,delta_32,delta_33,delta_34,delta_35,delta_36,delta_37,delta_38,delta_39,delta_40,delta_41,delta_42,delta_43,delta_44,delta_45,delta_46,delta_47,delta_48,delta_49,delta_50,delta_51,delta_52,delta_53,delta_54,delta_55,delta_56,delta_57,delta_58,delta_59,delta_60,delta_61,delta_62,delta_63,delta_64,delta_65,delta_66,delta_67,delta_68,delta_69,delta_70,delta_71,delta_72,delta_73,delta_74,delta_75,delta_76,delta_77,delta_78,delta_79,delta_80,delta_81,delta_82,delta_83,delta_84,delta_85,delta_86,delta_87
0,1,1,109088.0,93740.0,91600.0,91067.0,92129.0,90469.0,89488.0,90302.0,90889.0,92418.0,115549.0,98029.0,92372.0,91749.0,90771.0,90751.0,90984.0,95580.0,101892.0,105445.0,98683.0,153309.0,152352.0,136183.0,96863.0,92198.0,91291.0,90377.0,90077.0,89912.0,89743.0,90027.0,89560.0,89723.0,89474.0,89894.0,89422.0,89104.0,89020.0,90261.0,89824.0,89484.0,107494.0,95135.0,89444.0,90224.0,90528.0,90147.0,91175.0,90082.0,89611.0,89722.0,89599.0,91129.0,90149.0,107829.0,134442.0,150895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-15348.0,-2140.0,-533.0,1062.0,-1660.0,-981.0,814.0,587.0,1529.0,23131.0,-17520.0,-5657.0,-623.0,-978.0,-20.0,233.0,4596.0,6312.0,3553.0,-6762.0,54626.0,-957.0,-16169.0,-39320.0,-4665.0,-907.0,-914.0,-300.0,-165.0,-169.0,284.0,-467.0,163.0,-249.0,420.0,-472.0,-318.0,-84.0,1241.0,-437.0,-340.0,18010.0,-12359.0,-5691.0,780.0,304.0,-381.0,1028.0,-1093.0,-471.0,111.0,-123.0,1530.0,-980.0,17680.0,26613.0,16453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,157754.0,97640.0,91697.0,91867.0,92234.0,92036.0,91659.0,91801.0,93491.0,93063.0,92439.0,91452.0,91523.0,91408.0,91232.0,91112.0,91209.0,125300.0,121930.0,108094.0,108456.0,95396.0,121351.0,135301.0,97154.0,93808.0,93104.0,92280.0,92356.0,91390.0,91188.0,92289.0,91734.0,90967.0,90959.0,90756.0,90685.0,90269.0,89941.0,90031.0,89982.0,89950.0,89748.0,107500.0,94324.0,89421.0,88968.0,88283.0,88373.0,88697.0,89476.0,91178.0,91204.0,91444.0,91730.0,112621.0,128962.0,132652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-60114.0,-5943.0,170.0,367.0,-198.0,-377.0,142.0,1690.0,-428.0,-624.0,-987.0,71.0,-115.0,-176.0,-120.0,97.0,34091.0,-3370.0,-13836.0,362.0,-13060.0,25955.0,13950.0,-38147.0,-3346.0,-704.0,-824.0,76.0,-966.0,-202.0,1101.0,-555.0,-767.0,-8.0,-203.0,-71.0,-416.0,-328.0,90.0,-49.0,-32.0,-202.0,17752.0,-13176.0,-4903.0,-453.0,-685.0,90.0,324.0,779.0,1702.0,26.0,240.0,286.0,20891.0,16341.0,3690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,102659.0,92091.0,91148.0,89660.0,89468.0,89643.0,90181.0,91501.0,92315.0,90219.0,93984.0,91984.0,88519.0,88428.0,88553.0,120177.0,97629.0,93148.0,103569.0,102455.0,105705.0,155699.0,151994.0,141534.0,98511.0,91336.0,91444.0,90439.0,90328.0,90032.0,89822.0,89350.0,89152.0,89059.0,89294.0,89878.0,89801.0,89737.0,89748.0,90112.0,90626.0,88957.0,88728.0,106312.0,95294.0,89168.0,88506.0,87706.0,88803.0,90277.0,91532.0,91400.0,94441.0,96161.0,93169.0,123117.0,116092.0,150911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10568.0,-943.0,-1488.0,-192.0,175.0,538.0,1320.0,814.0,-2096.0,3765.0,-2000.0,-3465.0,-91.0,125.0,31624.0,-22548.0,-4481.0,10421.0,-

In [37]:
#Se filtran variables delta_:
lap_pivot[lap_pivot.filter(regex='delta', axis=1).columns.to_list()]

lap,delta_2,delta_3,delta_4,delta_5,delta_6,delta_7,delta_8,delta_9,delta_10,delta_11,delta_12,delta_13,delta_14,delta_15,delta_16,delta_17,delta_18,delta_19,delta_20,delta_21,delta_22,delta_23,delta_24,delta_25,delta_26,delta_27,delta_28,delta_29,delta_30,delta_31,delta_32,delta_33,delta_34,delta_35,delta_36,delta_37,delta_38,delta_39,delta_40,delta_41,delta_42,delta_43,delta_44,delta_45,delta_46,delta_47,delta_48,delta_49,delta_50,delta_51,delta_52,delta_53,delta_54,delta_55,delta_56,delta_57,delta_58,delta_59,delta_60,delta_61,delta_62,delta_63,delta_64,delta_65,delta_66,delta_67,delta_68,delta_69,delta_70,delta_71,delta_72,delta_73,delta_74,delta_75,delta_76,delta_77,delta_78,delta_79,delta_80,delta_81,delta_82,delta_83,delta_84,delta_85,delta_86,delta_87
0,-15348.0,-2140.0,-533.0,1062.0,-1660.0,-981.0,814.0,587.0,1529.0,23131.0,-17520.0,-5657.0,-623.0,-978.0,-20.0,233.0,4596.0,6312.0,3553.0,-6762.0,54626.0,-957.0,-16169.0,-39320.0,-4665.0,-907.0,-914.0,-300.0,-165.0,-169.0,284.0,-467.0,163.0,-249.0,420.0,-472.0,-318.0,-84.0,1241.0,-437.0,-340.0,18010.0,-12359.0,-5691.0,780.0,304.0,-381.0,1028.0,-1093.0,-471.0,111.0,-123.0,1530.0,-980.0,17680.0,26613.0,16453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-60114.0,-5943.0,170.0,367.0,-198.0,-377.0,142.0,1690.0,-428.0,-624.0,-987.0,71.0,-115.0,-176.0,-120.0,97.0,34091.0,-3370.0,-13836.0,362.0,-13060.0,25955.0,13950.0,-38147.0,-3346.0,-704.0,-824.0,76.0,-966.0,-202.0,1101.0,-555.0,-767.0,-8.0,-203.0,-71.0,-416.0,-328.0,90.0,-49.0,-32.0,-202.0,17752.0,-13176.0,-4903.0,-453.0,-685.0,90.0,324.0,779.0,1702.0,26.0,240.0,286.0,20891.0,16341.0,3690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-10568.0,-943.0,-1488.0,-192.0,175.0,538.0,1320.0,814.0,-2096.0,3765.0,-2000.0,-3465.0,-91.0,125.0,31624.0,-22548.0,-4481.0,10421.0,-1114.0,3250.0,49994.0,-3705.0,-10460.0,-43023.0,-7175.0,108.0,-1005.0,-111.0,-296.0,-210.0,-472.0,-198.0,-93.0,235.0,584.0,-77.0,-64.0,11.0,364.0,514.0,-1669.0,-229.0,17584.0,-11018.0,-6126.0,-662.0,-800.0,1097.0,1474.0,1255.0,-132.0,3041.0,1720.0,-2992.0,29948.0,-7025.0,34819.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-14384.0,-2808.0,939.0,-1283.0,-275.0,127.0,-393.0,-207.0,486.0,-625.0,-422.0,-19.0,411.0,-196.0,145.0,-117.0,1752.0,38628.0,-20115.0,-15309.0,57343.0,-755.0,-16723.0,-37727.0,-5928.0,-697.0,-444.0,-176.0,-463.0,-486.0,347.0,-309.0,-218.0,248.0,-175.0,239.0,-34.0,692.0,670.0,-733.0,378.0,-521.0,-49.0,-21.0,-157.0,112.0,-182.0,-238.0,-244.0,17852.0,-10820.0,-7771.0,12.0,447.0,34946.0,-5972.0,32300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-12165.0,-50.0,-1732.0,-800.0,-229.0,134.0,116.0,792.0,1348.0,-1509.0,539.0,-388.0,197.0,-556.0,-38.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,3959.0,4111.0,-28903.0,-329.0,205.0,253.0,152.0,31.0,225.0,128.0,-413.0,210.0,169.0,183.0,-2.0,1081.0,395.0,131.0,-804.0,3678.0,14192.0,-19331.0,-397.0,-27.0,-101.0,582.0,-146.0,-363.0,250.0,290.0,-799.0,5712.0,-3316.0,76.0,-1033.0,-522.0,-313.0,-175.0,2353.0,17598.0,-21514.0,-310.0,383.0,161.0,275.0,25.0,-34.0,135.0,-355.0,2419.0,-1973.0,538.0,1306.0,-383.0,-1030.0,652.0,82.0,-111.0,-75.0,24

In [40]:
#Se filtran variables delta_:
lap_pivot['delta_total'] = lap_pivot[lap_pivot.filter(regex='delta', axis=1).columns.to_list()].sum(axis=1)
lap_pivot

lap,raceId,driverId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,delta_2,delta_3,delta_4,delta_5,delta_6,delta_7,delta_8,delta_9,delta_10,delta_11,delta_12,delta_13,delta_14,delta_15,delta_16,delta_17,delta_18,delta_19,delta_20,delta_21,delta_22,delta_23,delta_24,delta_25,delta_26,delta_27,delta_28,delta_29,delta_30,delta_31,delta_32,delta_33,delta_34,delta_35,delta_36,delta_37,delta_38,delta_39,delta_40,delta_41,delta_42,delta_43,delta_44,delta_45,delta_46,delta_47,delta_48,delta_49,delta_50,delta_51,delta_52,delta_53,delta_54,delta_55,delta_56,delta_57,delta_58,delta_59,delta_60,delta_61,delta_62,delta_63,delta_64,delta_65,delta_66,delta_67,delta_68,delta_69,delta_70,delta_71,delta_72,delta_73,delta_74,delta_75,delta_76,delta_77,delta_78,delta_79,delta_80,delta_81,delta_82,delta_83,delta_84,delta_85,delta_86,delta_87,delta_total
0,1,1,109088.0,93740.0,91600.0,91067.0,92129.0,90469.0,89488.0,90302.0,90889.0,92418.0,115549.0,98029.0,92372.0,91749.0,90771.0,90751.0,90984.0,95580.0,101892.0,105445.0,98683.0,153309.0,152352.0,136183.0,96863.0,92198.0,91291.0,90377.0,90077.0,89912.0,89743.0,90027.0,89560.0,89723.0,89474.0,89894.0,89422.0,89104.0,89020.0,90261.0,89824.0,89484.0,107494.0,95135.0,89444.0,90224.0,90528.0,90147.0,91175.0,90082.0,89611.0,89722.0,89599.0,91129.0,90149.0,107829.0,134442.0,150895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-15348.0,-2140.0,-533.0,1062.0,-1660.0,-981.0,814.0,587.0,1529.0,23131.0,-17520.0,-5657.0,-623.0,-978.0,-20.0,233.0,4596.0,6312.0,3553.0,-6762.0,54626.0,-957.0,-16169.0,-39320.0,-4665.0,-907.0,-914.0,-300.0,-165.0,-169.0,284.0,-467.0,163.0,-249.0,420.0,-472.0,-318.0,-84.0,1241.0,-437.0,-340.0,18010.0,-12359.0,-5691.0,780.0,304.0,-381.0,1028.0,-1093.0,-471.0,111.0,-123.0,1530.0,-980.0,17680.0,26613.0,16453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41807.0
1,1,2,157754.0,97640.0,91697.0,91867.0,92234.0,92036.0,91659.0,91801.0,93491.0,93063.0,92439.0,91452.0,91523.0,91408.0,91232.0,91112.0,91209.0,125300.0,121930.0,108094.0,108456.0,95396.0,121351.0,135301.0,97154.0,93808.0,93104.0,92280.0,92356.0,91390.0,91188.0,92289.0,91734.0,90967.0,90959.0,90756.0,90685.0,90269.0,89941.0,90031.0,89982.0,89950.0,89748.0,107500.0,94324.0,89421.0,88968.0,88283.0,88373.0,88697.0,89476.0,91178.0,91204.0,91444.0,91730.0,112621.0,128962.0,132652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-60114.0,-5943.0,170.0,367.0,-198.0,-377.0,142.0,1690.0,-428.0,-624.0,-987.0,71.0,-115.0,-176.0,-120.0,97.0,34091.0,-3370.0,-13836.0,362.0,-13060.0,25955.0,13950.0,-38147.0,-3346.0,-704.0,-824.0,76.0,-966.0,-202.0,1101.0,-555.0,-767.0,-8.0,-203.0,-71.0,-416.0,-328.0,90.0,-49.0,-32.0,-202.0,17752.0,-13176.0,-4903.0,-453.0,-685.0,90.0,324.0,779.0,1702.0,26.0,240.0,286.0,20891.0,16341.0,3690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-25102.0
2,1,3,102659.0,92091.0,91148.0,89660.0,89468.0,89643.0,90181.0,91501.0,92315.0,90219.0,93984.0,91984.0,88519.0,88428.0,88553.0,120177.0,97629.0,93148.0,103569.0,102455.0,105705.0,155699.0,151994.0,141534.0,98511.0,91336.0,91444.0,90439.0,90328.0,90032.0,89822.0,89350.0,89152.0,89059.0,89294.0,89878.0,89801.0,89737.0,89748.0,90112.0,90626.0,88957.0,88728.0,106312.0,95294.0,89168.0,88506.0,87706.0,88803.0,90277.0,91532.0,91400.0,94441.0,96161.0,93169.0,123117.0,116092.0,150911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10568.0,-943.0,-1488.0,-192.0,175.0,538.0,1320.0,814.0,-2096.0,3765.0,-2000.0,-3465.0,-91.0,125.0,31624